In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import dateutil
from sklearn import svm
from sklearn import preprocessing
import timeit
import math
import time


whole_start_time = time.time()

In [2]:
#start_read = time.time()
#df = pd.read_csv("./train_sample.csv")
#print(time.time() - start_read)
#df

In [3]:
# get data in basic table form
start_read = time.time()
df = pd.read_csv("./train_downsampled.csv")
print(time.time() - start_read)
df

2.6225178241729736


/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,ip,app,device,os,channel,click_time,attributed_time,is_attributed
0,77656023,19507,26,1,15,266,2017-11-08 02:37:14,NaN,0
1,119537435,98186,12,1,17,105,2017-11-08 15:12:05,NaN,0
2,2815648,19873,22,1,11,116,2017-11-06 17:19:31,NaN,0
3,91573341,44645,2,1,6,205,2017-11-08 06:52:22,NaN,0
4,2624883,143561,2,2,56,477,2017-11-06 17:11:41,NaN,0
5,22561747,80402,9,1,22,215,2017-11-07 04:00:23,NaN,0
6,80105888,20533,12,1,13,178,2017-11-08 03:18:46,NaN,0
7,128354150,50347,3,1,19,280,2017-11-08 22:06:28,NaN,0
8,58465522,71373,9,1,6,215,2017-11-07 15:32:32,NaN,0
9,55599180,134601,12,1,23,245,2017-11-07 14:35:24,NaN,0


### Time to Attribution, hour of day, day of week, minute of hour

In [ ]:
df['click_time_ds'] = df.apply(lambda row: datetime.strptime(row['click_time'], '%Y-%m-%d %H:%M:%S'), axis=1)
df['time_to_attribution'] = df.apply(lambda row: datetime.strptime(row['attributed_time'], '%Y-%m-%d %H:%M:%S') - row['click_time_ds'] if isinstance(row['attributed_time'], str) else '', axis=1)

df[['hour_of_day', 'day_of_week', 'minute_of_hour']] = df.apply(lambda row: pd.Series([(row['click_time_ds'].hour + 8) % 24,
                                           row['click_time_ds'].weekday(),
                                           row['click_time_ds'].minute]), axis=1)

print(df['day_of_week'].max())
print(df['day_of_week'].min())
print(df['hour_of_day'].max())
print(df['hour_of_day'].min())
print(df['minute_of_hour'].max())
print(df['minute_of_hour'].min())

### Click Frequency

In [ ]:
df['click_time_dt'] = df.apply(lambda row: (dateutil.parser.parse(row['click_time'], dayfirst=True) - datetime.utcfromtimestamp(0)).total_seconds() , axis=1)


print(df['ip'].nunique())
f = {'click_time_dt':['min','max', 'count']}
ip_click_time_data = df.groupby('ip').agg(f)
ip_click_time_data




In [ ]:
# calculating click_frequency and statistics

freq_dict = {}
for index, row in ip_click_time_data.iterrows():
    # skip if only 1 click occurred
    if row['click_time_dt']['count'] == 1 or row['click_time_dt']['max'] == row['click_time_dt']['min']:
        freq_dict[index] = 0
    else:
        freq_dict[index] = row['click_time_dt']['count'] / (row['click_time_dt']['max'] - row['click_time_dt']['min'])
        
df['click_freq_for_ip'] = df.apply(lambda row: (freq_dict[row["ip"]]) , axis=1)
df.sort_values(by="ip")


In [ ]:
# data analysis on frequency list
freqs = list(freq_dict.values())
average_freq = sum(freqs)/len(freq_dict)
average_period = 1/average_freq
arr = np.array(freqs)
stdev = np.std(arr)
outliers = arr[(arr - np.mean(arr)) > 2 * np.std(arr)]
print(outliers)
#np.max(arr)

### Click time delta


In [ ]:

def hash_id(row, attributes):
    return ''.join(str(row[a]) for a in attributes)


def click_time_delta(row, attributes, store):
    uid = hash_id(row, attributes)
    prev_click_time = store.get(uid)
    store[uid] = row["click_time_dt"]
    if prev_click_time:
        return abs(row["click_time_dt"] - prev_click_time)
    else:
        return row["click_time_dt"]

df = df.sort_values(by='click_time_dt')

ip_prev_click_time = {}
df['prev_click_time_delta_for_ip'] = df.apply(lambda row: click_time_delta(row, ["ip"], ip_prev_click_time) , axis=1)

ip_os_device_prev_click_time = {}
df['prev_click_time_delta_for_ip_os_device'] = df.apply(lambda row: click_time_delta(row, ["ip", "os", "device"], ip_os_device_prev_click_time) , axis=1)

ip_app_os_device_prev_click_time = {}
df['prev_click_time_delta_for_ip_app_os_device'] = df.apply(lambda row: click_time_delta(row, ["ip", "app", "os", "device"], ip_app_os_device_prev_click_time) , axis=1)

ip_app_prev_click_time = {}
df['prev_click_time_delta_for_ip_app'] = df.apply(lambda row: click_time_delta(row, ["ip", "app"], ip_app_prev_click_time) , axis=1)



In [ ]:
df = df.sort_values(by='click_time_dt', ascending=False)

ip_next_click_time = {}
df['next_click_time_delta_for_ip'] = df.apply(lambda row: click_time_delta(row, ["ip"], ip_next_click_time) , axis=1)

ip_os_device_next_click_time = {}
df['next_click_time_delta_for_ip_os_device'] = df.apply(lambda row: click_time_delta(row, ["ip", "os", "device"], ip_os_device_next_click_time) , axis=1)

ip_app_os_device_next_click_time = {}
df['next_click_time_delta_for_ip_app_os_device'] = df.apply(lambda row: click_time_delta(row, ["ip", "app", "os", "device"], ip_app_os_device_next_click_time) , axis=1)

ip_app_next_click_time = {}
df['next_click_time_delta_for_ip_app'] = df.apply(lambda row: click_time_delta(row, ["ip", "app"], ip_app_next_click_time) , axis=1)

df

### Attribution ratio and Clicks per attribute

In [ ]:
num_clicks_per_ip = df.groupby('ip').agg({'click_time_dt':['count'], 'is_attributed': ['sum']})
num_clicks_per_ip_os_device = df.groupby(['ip', 'os', 'device']).agg({'click_time_dt':['count'], 'is_attributed': ['sum']})
num_clicks_per_app_channel = df.groupby(['channel', 'app']).agg({'click_time_dt':['count'], 'is_attributed': ['sum']})
num_clicks_per_app = df.groupby('app').agg({'click_time_dt':['count'], 'is_attributed': ['sum']})
num_clicks_per_ip_app = df.groupby(['ip','app']).agg({'click_time_dt':['count'], 'is_attributed': ['sum']})

def create_dict(grouped_data):
    ret_dict = {}
    for index, row in grouped_data.iterrows():
        # skip if only 1 click occurred
        ret_dict[index] = {'click_count': row['click_time_dt']['count'], 'attributions': row['is_attributed']['sum']}
    return ret_dict

start = time.time()
num_clicks_per_ip_dict = create_dict(num_clicks_per_ip)
end = time.time()
print(end - start)

start = time.time()
num_clicks_per_ip_os_device_dict = create_dict(num_clicks_per_ip_os_device)
end = time.time()
print(end - start)

start = time.time()
num_clicks_per_app_channel_dict = create_dict(num_clicks_per_app_channel)
end = time.time()
print(end - start)

start = time.time()
num_clicks_per_app_dict = create_dict(num_clicks_per_app)
end = time.time()
print(end - start)

start = time.time()
num_clicks_per_ip_app_dict = create_dict(num_clicks_per_ip_app)
end = time.time()
print(end - start)


In [ ]:
print(int(num_clicks_per_ip_os_device.loc[[(27, 19, 1)]]['click_time_dt']['count']))
start = time.time()
df['num_clicks_per_ip'] = df.apply(lambda row: int(num_clicks_per_ip_dict[row['ip']]['click_count']), axis=1)
end = time.time()
print(end - start)

df['num_clicks_per_ip_app'] = df.apply(lambda row: int(num_clicks_per_ip_app_dict[(row['ip'], row['app'])]['click_count']) , axis=1)

df['num_clicks_per_ip_os_device'] = df.apply(lambda row: int(num_clicks_per_ip_os_device_dict[(row['ip'], row['os'], row['device'])]['click_count']) , axis=1)
df['num_clicks_per_app_channel'] = df.apply(lambda row: int(num_clicks_per_app_channel_dict[(row['channel'], row['app'])]['click_count']) , axis=1)
df['num_clicks_per_app'] = df.apply(lambda row: int(num_clicks_per_app_dict[row['app']]['click_count']) , axis=1)

df

In [ ]:

df['attribution_ratio_ip'] = df.apply(lambda row: float(num_clicks_per_ip_dict[row['ip']]['attributions'])/row['num_clicks_per_ip'], axis=1)
df['attribution_ratio_ip_os_device'] = df.apply(lambda row: float(num_clicks_per_ip_os_device_dict[(row['ip'], row['os'], row['device'])]['attributions'])/row['num_clicks_per_ip_os_device'] , axis=1)
df['attribution_ratio_app'] = df.apply(lambda row: float(num_clicks_per_app_dict[row['app']]['attributions'])/row['num_clicks_per_app'] , axis=1)
df['attribution_ratio_ip_app'] = df.apply(lambda row: float(num_clicks_per_ip_app_dict[(row['ip'], row['app'])]['attributions'])/row['num_clicks_per_ip_app'] , axis=1)
df['attribution_ratio_app_channel'] = df.apply(lambda row: float(num_clicks_per_app_channel_dict[(row['channel'], row['app'])]['attributions'])/row['num_clicks_per_app_channel'] , axis=1)

df


In [ ]:

def hash_id(row, attributes):
    return ''.join(str(row[a]) for a in attributes)


def click_times_in_hour(row, attributes, store):
    uid = hash_id(row, attributes)
    cts = store.get(uid)
    
    if cts:
        for ct in cts:
            if row["click_time_dt"] - ct > 60*60:
                cts.remove(ct)
        cts.append(row["click_time_dt"])
        return len(cts)
    else:
        store[uid] = [row["click_time_dt"]]
        return 0

df = df.sort_values(by='click_time_dt')

ip_prev_click_times = {}
df['num_clicks_in_hour_for_ip'] = df.apply(lambda row: click_times_in_hour(row, ["ip"], ip_prev_click_times) , axis=1)

ip_os_device_prev_click_times = {}
df['num_clicks_in_hour_for_ip_os_device'] = df.apply(lambda row: click_times_in_hour(row, ["ip", "os", "device"], ip_os_device_prev_click_times) , axis=1)

ip_app_os_device_prev_click_times = {}
df['num_clicks_in_hour_for_ip_app_os_device'] = df.apply(lambda row: click_times_in_hour(row, ["ip", "app", "os", "device"], ip_app_os_device_prev_click_times) , axis=1)

ip_app_prev_click_times = {}
df['num_clicks_in_hour_for_ip_app'] = df.apply(lambda row: click_times_in_hour(row, ["ip", "app"], ip_app_prev_click_times) , axis=1)
df

### Given IP, get number unique things

In [ ]:
## TODO add this
#number of unique os for each ip
#df1 = df[['ip','os']].groupby(['ip']).agg({'os': pd.Series.nunique})

#numer of unique apps that each ip clicked on
#df2 = df[['ip','app']].groupby(['ip']).agg({'app': pd.Series.nunique})

#number of unique (os,device) pairs per ip
#df3 = df[['ip','os','device','click_time']].groupby(['ip']).agg({'click_time': pd.Series.nunique})

#number of unique (app,channel) pairs per ip
#df4 = df[['ip','app','channel','click_time']].groupby(['ip']).agg({'click_time': pd.Series.nunique})

#df3

In [ ]:
print("TOTAL TIME")
print(time.time() - whole_start_time)

In [ ]:
print(len(df.columns))
print(len(df))
print(df.columns)
df.to_csv("feature_engineered_data.csv")


## CREATE TEST FEATURE ENGINEERING

In [ ]:
start_read = time.time()
test_df = pd.read_csv("./test.csv")
print(time.time() - start_read)
test_df

In [ ]:
start = time.time()
test_df['click_time_ds'] = test_df.apply(lambda row: datetime.strptime(row['click_time'], '%Y-%m-%d %H:%M:%S'), axis=1)
print(time.time() - start)

start = time.time()
test_df[['hour_of_day', 'day_of_week', 'minute_of_hour']] = test_df.apply(lambda row: pd.Series([(row['click_time_ds'].hour + 8) % 24,
                                           row['click_time_ds'].weekday(),
                                           row['click_time_ds'].minute]), axis=1)

print(time.time() - start)

start = time.time()
test_df['click_time_dt'] = test_df.apply(lambda row: (dateutil.parser.parse(row['click_time'], dayfirst=True) - datetime.utcfromtimestamp(0)).total_seconds() , axis=1)
print(time.time() - start)

start = time.time()
print(time.time() - start)



In [ ]:
test_df = test_df.sort_values(by='click_time_dt')

test_df['prev_click_time_delta_for_ip'] = test_df.apply(lambda row: ip_prev_click_time.get(hash_id(row, ['ip'])) , axis=1)

test_df['prev_click_time_delta_for_ip_os_device'] = test_df.apply(lambda row: ip_os_device_prev_click_time.get(hash_id(row, ['ip','os', 'device'])) , axis=1)

test_df['prev_click_time_delta_for_ip_app_os_device'] = test_df.apply(lambda row: ip_app_os_device_prev_click_time.get(hash_id(row, ['ip','app', 'os', 'device'])) , axis=1)

test_df['prev_click_time_delta_for_ip_app'] = test_df.apply(lambda row: ip_app_prev_click_time.get(hash_id(row, ['ip','app'])) , axis=1)



In [ ]:
test_df = test_df.sort_values(by='click_time_dt', ascending=False)

test_df['next_click_time_delta_for_ip'] = test_df.apply(lambda row: ip_next_click_time.get(hash_id(row, ['ip'])) , axis=1)

test_df['next_click_time_delta_for_ip_os_device'] = test_df.apply(lambda row: ip_os_device_next_click_time.get(hash_id(row, ['ip', 'os', 'device'])) , axis=1)

test_df['next_click_time_delta_for_ip_app_os_device'] = test_df.apply(lambda row: ip_app_os_device_next_click_time.get(hash_id(row, ['ip', 'app', 'os', 'device'])) , axis=1)

test_df['next_click_time_delta_for_ip_app'] = test_df.apply(lambda row: ip_app_next_click_time.get(hash_id(row, ['ip', 'app'])) , axis=1)



In [ ]:
start = time.time()
test_df['num_clicks_per_ip'] = test_df.apply(lambda row: int(num_clicks_per_ip_dict[row['ip']]['click_count']), axis=1)
end = time.time()
print(end - start)

test_df['num_clicks_per_ip_app'] = test_df.apply(lambda row: int(num_clicks_per_ip_app_dict[(row['ip'], row['app'])]['click_count']) , axis=1)

test_df['num_clicks_per_ip_os_device'] = test_df.apply(lambda row: int(num_clicks_per_ip_os_device_dict[(row['ip'], row['os'], row['device'])]['click_count']) , axis=1)
test_df['num_clicks_per_app_channel'] = test_df.apply(lambda row: int(num_clicks_per_app_channel_dict[(row['channel'], row['app'])]['click_count']) , axis=1)
test_df['num_clicks_per_app'] = test_df.apply(lambda row: int(num_clicks_per_app_dict[row['app']]['click_count']) , axis=1)


In [ ]:
test_df['attribution_ratio_ip'] = test_df.apply(lambda row: float(num_clicks_per_ip_dict[row['ip']]['attributions'])/row['num_clicks_per_ip'], axis=1)
test_df['attribution_ratio_ip_os_device'] = test_df.apply(lambda row: float(num_clicks_per_ip_os_device_dict[(row['ip'], row['os'], row['device'])]['attributions'])/row['num_clicks_per_ip_os_device'] , axis=1)
test_df['attribution_ratio_app'] = test_df.apply(lambda row: float(num_clicks_per_app_dict[row['app']]['attributions'])/row['num_clicks_per_app'] , axis=1)
test_df['attribution_ratio_ip_app'] = test_df.apply(lambda row: float(num_clicks_per_ip_app_dict[(row['ip'], row['app'])]['attributions'])/row['num_clicks_per_ip_app'] , axis=1)
test_df['attribution_ratio_app_channel'] = test_df.apply(lambda row: float(num_clicks_per_app_channel_dict[(row['channel'], row['app'])]['attributions'])/row['num_clicks_per_app_channel'] , axis=1)


In [ ]:
test_df = test_df.sort_values(by='click_time_dt')

test_df['num_clicks_in_hour_for_ip'] = test_df.apply(lambda row: ip_prev_click_times.get(hash_id(row, ['ip'])) , axis=1)

test_df['num_clicks_in_hour_for_ip_os_device'] = test_df.apply(lambda row: ip_os_device_prev_click_times.get(hash_id(row, ['ip', 'os', 'device'])) , axis=1)

test_df['num_clicks_in_hour_for_ip_app_os_device'] = test_df.apply(lambda row: ip_app_os_device_prev_click_times.get(hash_id(row, ['ip', 'app', 'os', 'device'])) , axis=1)

test_df['num_clicks_in_hour_for_ip_app'] = test_df.apply(lambda row: ip_app_prev_click_times.get(hash_id(row, ['ip', 'app'])) , axis=1)


In [ ]:
print(len(test_df.columns))
print(len(test_df))
print(test_df.columns)
test_df = test_df.drop(['Unnamed: 0'])
test_df.to_csv("feature_engineered_test.csv")